In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

In [13]:
def ParseBoiler(filename):
    
    dfOrig = pd.read_csv('/dfs/scratch0/bosch/BG-Data_Part11/2029718103746674690.csv',
                     parse_dates=[['Date','Time']], delimiter='\t')
    dfOrig2 = dfOrig.drop([0]) #TODO - remove this?
    dfOrig2['Date_Time'] = pd.to_datetime(dfOrig2['Date_Time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')
    
    #Only select the columns we care about
    dfSubset = dfOrig2[['Date_Time', 'Operating_status:_Error_Locking', 'Operating_status:_Error_Blocking',
                   'Actual_Power', 'Number_of_burner_starts', 'Operating_status:_Central_heating_active',
                  'Operating_status:_Hot_water_active', 'Operating_status:_Flame', 'Relay_status:_Gasvalve',
                  'Relay_status:_Fan', 'Relay_status:_Ignition', 'Relay_status:_internal_3-way-valve',
                  'Relay_status:_HW_circulation_pump', 'Supply_temperature_(primary_flow_temperature)',
                  'Maximum_supply_(primary_flow)_temperature', 'Hot_water_temperature_setpoint', 
                  'Hot_water_outlet_temperature', 'Actual_flow_rate_turbine', 'Fan_speed']]
    
    #Fill in missing entries
    df = dfSubset.fillna(method = 'ffill')
    
    print "Successfully loaded", dfOrig
    
    Positives = df.loc[(df['Operating_status:_Error_Locking'] == 1) 
                       & (df['Operating_status:_Error_Locking'].shift(1) == 0)]
    
      
    for index, row in Positives.iterrows():
        #For each Positive example...
        eventTime = row['Date_Time']
        startTime = eventTime - pd.Timedelta('7 days')
        dfPos = df[(df['Date_Time'] >= startTime) & (df['Date_Time'] < eventTime)]
        
        print dfPos.shape
        
        sensorList = ['Actual_Power', 'Number_of_burner_starts', 'Operating_status:_Central_heating_active',
                  'Operating_status:_Hot_water_active', 'Operating_status:_Flame', 'Relay_status:_Gasvalve',
                  'Relay_status:_Fan', 'Relay_status:_Ignition', 'Relay_status:_internal_3-way-valve',
                  'Relay_status:_HW_circulation_pump', 'Supply_temperature_(primary_flow_temperature)',
                  'Maximum_supply_(primary_flow)_temperature', 'Hot_water_temperature_setpoint', 
                  'Hot_water_outlet_temperature', 'Actual_flow_rate_turbine', 'Fan_speed']
        
        for sensor in sensorList:
            print "Getting data for", sensor
            
            rawData = np.zeros((168,5))
            for j in range(168):
                lastTime = startTime + j*pd.Timedelta('1 hour')
                currTime = startTime + (j+1)*pd.Timedelta('1 hour')
                
                dfHourly = dfPos[(dfPos['Date_Time'] >= lastTime) & (dfPos['Date_Time'] < currTime)]

                description = dfHourly[sensor].describe()
                rawData[j,0] = description['min']
                rawData[j,1] = description['max']
                rawData[j,2] = description['mean']
                rawData[j,3] = description['std']
                rawData[j,4] = description['count']

                #TODO - FENG'S CODE GOES HERE!
        

In [ ]:
#Parse ALL Boilers - TODO - unzip other folders too!
for filename in os.listdir('/dfs/scratch0/bosch/BG-Data_Part11/'):
     ParseBoiler(filename)

In [ ]:
#Parse just one boiler (for testing...)
ParseBoiler('/dfs/scratch0/bosch/BG-Data_Part11/2029718103746674690.csv')